In [1]:
%load_ext autoreload
%autoreload 2

import unicodecsv as csv
import datetime
import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article
import multiprocessing 
import pandas as pd

In [2]:
%%time
# Compiling papers with using rss information, producing newspaper objects:
all_cnn_papers = helpers.compile_papers(rss_feeds_dict=helpers.CNN_LINKS_DICT)
all_bbc_papers = helpers.compile_papers(rss_feeds_dict=helpers.BBC_LINKS_DICT)
all_fox_papers = helpers.compile_papers(rss_feeds_dict=helpers.FOX_LINKS_DICT)
all_rt_papers = helpers.compile_papers(rss_feeds_dict=helpers.RT_LINKS_DICT)
# No msnbc rss, so can't used this method with msnbc

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_world.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_us.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_allpolitics.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_tech.rss
[Parse lxml ERR] U

In [3]:
%%time
# Download articles, i.e. scrapeoff text and other data:
helpers.art_multithread_downloads(papers=all_cnn_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_bbc_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_fox_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_rt_papers, threads_per_source=2)

CPU times: user 35.9 s, sys: 6.53 s, total: 42.4 s
Wall time: 2min 25s


In [4]:
master_paper_list = [all_cnn_papers, 
                     all_bbc_papers,
                     all_fox_papers,
                     all_rt_papers]

In [5]:
%%time
# Parse all articles:
for paper_list in master_paper_list:
    for paper in paper_list:
        print("Paper:", paper.url)
        paper.parse_articles()

('Paper:', 'http://rss.cnn.com/rss/cnn_topstories.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33434
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 33437
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 36868
  " Skipping tag %s" % (size,

/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 37122
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 0. Skipping tag 36880
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 0. Skipping tag 36881
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7 bytes but only got 0. Skipping tag 36882
  " Skipping tag %s" % (size, l

('Paper:', 'http://rss.cnn.com/rss/cnn_world.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 28 bytes but only got 0. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12 bytes but only got 0. Skipping tag 42032
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 0. Skipping tag 42036
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
/Users/alexander/anaconda3/en

...

('Paper:', 'http://rss.cnn.com/rss/cnn_health.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 100. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 148. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 24 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 24 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size,

('Paper:', 'http://rss.cnn.com/rss/cnn_showbiz.rss')


/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 20. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 8. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 16. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 88. Skipping tag 37510
  " Skipping tag %s" %

('Paper:', 'http://rss.cnn.com/rss/cnn_travel.rss')
('Paper:', 'http://rss.cnn.com/rss/cnn_latest.rss')
('Paper:', 'http://feeds.bbci.co.uk/news/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/world/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/uk/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/business/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/technology/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/science_and_environment/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/entertainment_and_arts/rss.xml')
('Paper:', 'http://feeds.bbci.co.uk/news/health/rss.xml')
('Paper:', 'http://www.feeds.foxnews.com/foxnews/latest')
('Paper:', 'http://feeds.foxnews.com/foxnews/most-popular')
('Paper:', 'http://feeds.foxnews.com/foxnews/entertainment')
('Paper:', 'http://feeds.foxnews.com/foxnews/health')
('Paper:', 'http://feeds.foxnews.com/foxnews/section/lifestyle')
('Paper:', 'http://feeds.foxnews.com

In [7]:
def write_out_articles(newspaper_obj, filepath, writeout_type='conventional'):
    """
    TODO: finish this docstring
    :param newspaper_obj:
    :param filepath: str, filepath, WITHOUT the filename
    :param writeout_type: str, 'custom' or 'conventional'
    :return:
    """
    now = datetime.datetime.now()
    current_date = '{}_{}_{}'.format(str(now.year),
                                     str(now.month),
                                     str(now.day))
    if writeout_type == 'custom':
        csv_name = filepath + current_date + '_' + newspaper_obj.description \
                   + '.csv'
    else:
        csv_name = filepath + current_date + '_' + newspaper_obj.brand + '.csv'
    with open(csv_name, 'w') as outcsv:
        writer = csv.writer(outcsv, encoding='utf-8')
        if writeout_type == 'conventional':
            header = ['source_url', 'url', 'title', 'movies', 'text',
                      'keywords', 'meta_keywords', 'tags', 'authors',
                      'publish_date', 'summary', 'html', 'article_html',
                      'meta_description', 'meta_data', 'canonical_link']
        else:
            header = ['source_url', 'url', 'title', 'movies', 'text',
                             'keywords', 'meta_keywords', 'tags', 'authors',
                             'publish_date', 'summary', 'html', 'article_html',
                             'meta_description', 'meta_data', 'canonical_link',
                             'rss_title', 'rss_link', 'rss_id', 'rss_published',
                             'rss_published_parsed', 'rss_feedburner_origlink',
                             'paper_section_name']
        writer.writerow(header)
        for article in newspaper_obj.articles:
            source_url = article.source_url
            url = article.url
            title = article.title
            movies = article.movies
            text = article.text
            keywords = article.keywords
            meta_keywords = article.meta_keywords
            tags = article.tags
            authors = article.authors
            publish_date = article.published_date
            summary = article.summary
            html = article.html
            article_html = article.article_html
            meta_description = article.meta_description
            meta_data = article.meta_data
            canonical_link = article.canonical_link
            if writeout_type == 'custom':
                rss_title = article.additional_data['rss_title']
                rss_link = article.additional_data['rss_link']
                rss_id = article.additional_data['rss_id']
                rss_published = article.additional_data['rss_published']
                rss_published_parsed = \
                    article.additional_data['rss_published_parsed']
                rss_feedburner_origlink = \
                    article.additional_data['rss_feedburner_origlink']
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link, rss_title, rss_link,
                            rss_id, rss_published, rss_published_parsed,
                            rss_feedburner_origlink,
                            newspaper_obj.description]
            else:
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link]
            writer.writerow(data_row)

In [8]:
filepath = 'data/custom_data_pulls/'

In [9]:
%%time
for paper_list in master_paper_list:
    for paper in paper_list:
        write_out_articles(newspaper_obj=paper,
                                   filepath=filepath, 
                                   writeout_type='custom')

CPU times: user 10.4 s, sys: 2.06 s, total: 12.4 s
Wall time: 12.9 s


Check for files:

In [ ]:
df = pd.read_csv('data/custom_data_pulls/2019_2_3_rt_main_page.csv')

In [ ]:
df.text